# 🧠 Model Evolution & Evaluation

> **PM Accelerator Mission**: "By making industry-leading tools and education available to individuals from all backgrounds, we level the playing field for future PM leaders."

This notebook covers:
1. Model Architecture Comparison
2. Training Process
3. Evaluation Metrics
4. Final Model Deep Dive

## 1. Model Evolution Journey

### 📈 Performance Timeline

| Version | Model | MAE | Parameters | Training Time |
|---------|-------|-----|------------|---------------|
| V1.0 | MLP (3 layers) | ~4.5°C | ~50K | 5 min |
| V2.2 | LSTM (2 layers) | 2.05°C | ~100K | 30 min |
| V2.3 | Transformer (4 layers) | 2.05°C | ~200K | 15 min |
| V3.0 | Multivariate Transformer | 2.07°C | ~250K | 20 min |
| **V4.0** | **Advanced Transformer + GRN** | **2.00°C** | **1.3M** | **45 min** |

In [ ]:
import plotly.graph_objects as go

# Model evolution visualization
versions = ['V1 MLP', 'V2.2 LSTM', 'V2.3 Transformer', 'V3.0 Multivariate', 'V4.0 Advanced']
mae_scores = [4.5, 2.05, 2.05, 2.07, 2.00]
colors = ['#ff6b6b', '#ffd93d', '#6bcb77', '#4d96ff', '#00f296']

fig = go.Figure(data=[
    go.Bar(x=versions, y=mae_scores, marker_color=colors, text=[f'{m}°C' for m in mae_scores], textposition='outside')
])
fig.update_layout(
    title='📊 Model Performance Evolution (Lower is Better)',
    yaxis_title='MAE (°C)',
    template='plotly_dark',
    yaxis_range=[0, 5]
)
fig.add_hline(y=2.0, line_dash='dash', line_color='green', annotation_text='Target: 2.0°C')
fig.show()

## 2. Architecture Deep Dives

### V1: Multi-Layer Perceptron (MLP)

```
Input (20 features) → Dense(256, ReLU) → Dense(128, ReLU) → Dense(64, ReLU) → Output(7)
```

**Pros:**
- Simple, fast to train
- Easy to interpret

**Cons:**
- No temporal awareness
- Cannot capture sequences

---

### V2.2: LSTM (Long Short-Term Memory)

```
Input Sequence (30 days) → LSTM(128) → LSTM(128) → Dense(64) → Output(7)
```

**Innovation:**
- First model to use **30-day historical sequence**
- Remembers patterns across days

**Pros:**
- Captures temporal dependencies
- Good at short-term patterns

**Cons:**
- Slow training (sequential nature)
- Struggles with long-range dependencies

---

### V2.3: Transformer

```
Input (30 days) → Positional Encoding → Transformer Encoder (4 layers) → Output Head → 7 days
```

**Innovation:**
- **Self-Attention**: Every day can attend to every other day
- **Parallelizable**: Much faster training

**Pros:**
- Captures long-range dependencies
- Faster than LSTM

**Cons:**
- Treats all features equally (no importance weighting)

### V4.0: Advanced Transformer with GRN (Final Model)

```
Input (30 days × 25 features)
        ↓
┌───────────────────────┐
│ Gated Residual Network│  ← Learns to skip irrelevant inputs
└───────────────────────┘
        ↓
┌───────────────────────┐
│  Positional Encoding  │  ← Injects sequence order
└───────────────────────┘
        ↓
┌───────────────────────┐
│  Transformer Encoder  │  ← 6 layers, 8 attention heads
│   (Self-Attention)    │
└───────────────────────┘
        ↓
┌───────────────────────┐
│ Gated Residual Network│  ← Filters noise before output
└───────────────────────┘
        ↓
    7-Day Forecast
```

### What is a Gated Residual Network (GRN)?

The GRN is a key innovation that allows the model to **selectively ignore irrelevant features**.

```python
# Simplified GRN concept:
def grn(x):
    hidden = elu(fc1(x))
    output = fc2(hidden)
    gate = sigmoid(fc_gate(hidden))  # 0-1 importance weight
    return layer_norm(gate * output + skip(x))  # Skip connection!
```

**Key Benefits:**
1. **Feature Importance**: The gate learns which features matter
2. **Skip Connections**: If a feature is useless, it can be bypassed
3. **Noise Filtering**: Reduces impact of noisy weather data

## 3. Training Configuration

In [ ]:
# V4 Training Configuration
config = {
    'architecture': 'AdvancedWeatherTransformer',
    'd_model': 128,
    'num_heads': 8,
    'num_layers': 6,
    'dropout': 0.15,
    'sequence_length': 30,
    'prediction_length': 7,
    'features': 25,
    'parameters': '1,324,167',
    'loss_function': 'HuberLoss (delta=1.0)',
    'optimizer': 'AdamW (lr=0.001, weight_decay=0.02)',
    'scheduler': 'CosineAnnealingWarmRestarts (T_0=10)',
    'batch_size': 256,
    'epochs': 44,  # Early stopped from 150
    'gradient_clipping': 0.5
}

print("🔧 V4 Model Configuration:")
for k, v in config.items():
    print(f"   {k}: {v}")

### Why HuberLoss?

Standard MSE (Mean Squared Error) heavily penalizes outliers:
- If actual temp is 20°C and prediction is 50°C, MSE = 900!

HuberLoss combines MSE and MAE:
- For small errors: Behaves like MSE (smooth gradient)
- For large errors: Behaves like MAE (less penalty)

This makes training **robust to outliers** in weather data.

## 4. Evaluation Results

In [ ]:
# Per-day MAE for V4 model
days = ['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5', 'Day 6', 'Day 7']
mae_per_day = [1.97, 1.95, 1.94, 1.94, 1.99, 2.05, 2.14]

fig = go.Figure(data=[
    go.Scatter(x=days, y=mae_per_day, mode='lines+markers+text', 
               text=[f'{m}°C' for m in mae_per_day], textposition='top center',
               line=dict(color='#4facfe', width=3),
               marker=dict(size=12))
])
fig.update_layout(
    title='📈 MAE by Forecast Day (V4 Model)',
    xaxis_title='Forecast Day',
    yaxis_title='MAE (°C)',
    template='plotly_dark',
    yaxis_range=[1.5, 2.5]
)
fig.show()

print("\n📊 Key Observation:")
print("   - Days 1-4: Very accurate (~1.95°C)")
print("   - Days 5-7: Slight degradation (~2.1°C)")
print("   - This is expected: uncertainty grows with forecast horizon")

## 5. Key Learnings

### What Worked Well
1. **Sequence Modeling**: Moving from MLP to LSTM cut MAE in half
2. **Gated Residual Networks**: Allowed selective feature importance
3. **Real Weather Data**: Open-Meteo integration improved generalization
4. **Cyclical Encoding**: Sin/cos for time features

### What Didn't Work
1. **Adding more features naively**: V3 multivariate was WORSE than V2.3
   - Lesson: Need proper feature selection/gating
2. **Larger batch sizes**: Beyond 256 hurt convergence
3. **Very deep models (>8 layers)**: Overfitting increased

### Future Improvements
1. **Elevation Data**: Temperature drops ~6.5°C per 1000m altitude
2. **Ocean Proximity**: Coastal regions have more stable temperatures
3. **Real-time Weather API**: Use forecast data for initial conditions

---

## 🏁 Conclusion

We successfully built a **state-of-the-art weather forecasting system** that:
- Achieves **2.00°C MAE** (exceeded 2.5°C target)
- Works for **any location on Earth**
- Provides **real-time predictions** via web app
- Is **production-ready** with Docker deployment

The journey from V1 (4.5°C MAE) to V4 (2.0°C MAE) demonstrates the power of iterative model improvement and proper feature engineering.